In [20]:
import os
import re
import json
import time
import logging
import requests
import warnings
import kagglehub
import unicodedata
import pandas as pd
from edgar import *
from tqdm import tqdm
from urllib3 import Retry
from datetime import datetime
from kagglehub import KaggleDatasetAdapter
from requests.adapters import HTTPAdapter

set_identity("ansa ansa1019@gmail.com")
warnings.simplefilter("ignore")
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("request").setLevel(logging.ERROR)
request = requests.Session()
retries = Retry(
    total=3,  # 最多重試 5 次
    backoff_factor=1,  # 每次重試的延遲時間指數增長（1s, 2s, 4s, 8s...）
    status_forcelist=[403, 429, 500, 502, 503, 504],  # 針對這些 HTTP 狀態碼進行重試
)
request.mount("http://", HTTPAdapter(max_retries=retries))
request.mount("https://", HTTPAdapter(max_retries=retries))

df_companies = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS, "andrewmvd/sp-500-stocks", "sp500_companies.csv"
).drop_duplicates(subset=["Shortname"], keep="first")
companies = df_companies[df_companies["Country"] == "United States"]["Symbol"].values
with open("companies.json", "r", encoding="utf-8") as f:
    data = json.load(f)
missing_companies = [entry["ticker"] for entry in data["missing_companies"]]
find_cik = {entry["ticker"]: entry["cik"] for entry in data["find_cik"]}
companies = [c for c in companies if c not in missing_companies]
companies_range = 100
report_papers = "papers.json"
financials_file = "financials.csv"
search_queries = ["IT capability", "organizational resilience"]
report_item = {"paper_7": ["7"], "paper_17": ["1A", "7"]}  # "root_folder":["Items"]
report_year = [2014, 2023]  # 2014/1/1-2023/12/31
minlen = 1500
keyword_num = 30
keywords = {
    "IT capability": ["IT capability"],
    "organizational resilience": ["organizational resilience"],
}

roots = list(report_item.keys())
define_folder = "define/"
report_folder = {}
for root in roots:
    report_folder[root] = f"{root}/report/"

In [2]:
ticker_pattern = {
    "GE": {
        "start": r"^Management\W?s?\s*Discussion\s*And\s*Analysis\s*of\s*Financial\s*Condition\s*and\s*Results\s*of\s*Operations?.?\s*\(MD&A\)",
        "end": r"^Other\s*Financial\s*Data\.?\s*(?:Five-Year\s*Performance\s*graph|Selected\s*Financial\s*Data)",
    },
    "C": {
        "start": r"^Management’s\s+Discussion\s+And\s+Analysis\s+Of\s+Financial\s+Condition\s+And\s+Results\s+Of\s+Operations\nExecutive\s+Summary\s*$",
        "end": r"^\s*Capital\s+Resources\s*$",
    },
}

In [3]:
def save_to_file(content, path):
    with open(path, "w", encoding="utf-8") as file:
        file.write(content)


def normalize(text):
    def fix_linebreaks(match):
        return match.group(1) + " " + match.group(2)

    text = unicodedata.normalize("NFKD", text)
    substitutions = [
        (r"[’‘‛ʼ′＇ꞌʹ]", "'"),
        (r"[“”„‟〝〞＂]", '"'),
        (r"[–—―−]", "-"),
        (r"[\uFEFF]", ""),
        (r"[\u00A0\u2000-\u200B\u202F\u205F\u2060\u3000]", " "),
    ]
    for pattern, replacement in substitutions:
        text = re.sub(pattern, replacement, text)
    text = "\n".join(text.splitlines())
    text = re.sub(re.compile(r"[\s\n]*\n+[\s\n]*", re.I | re.M), "\n", text)
    text = text.strip()
    text = re.sub(r"([a-z,])\s*$\s*([a-z,])", fix_linebreaks, text, flags=re.I | re.M)
    remove_pattern = re.compile(r"^\s+|\xa0|&nbsp;|&#160;", re.I | re.M)
    text = re.sub(remove_pattern, "", text)
    clear_pattern = [
        r"^\s*[-]+\s*$",
        r"^\s*Table\s*of\s*Contents?\s*$",
        r"^\s*[^\n]*\|?[^\S\n]*\d{4}[^\S\n]*Form[^\S\n]*10-K[^\S\n]*\|?[^\S\n]*\d*\s*$",
    ]
    for x in clear_pattern:
        text = re.sub(re.compile(x, re.I | re.M), "", text)
    text = text.lower()
    return text


def find_mda(text, pattern_start, pattern_end):
    global minlen

    start_matches = list(re.finditer(pattern_start, text, re.I | re.M))
    end_matches = list(re.finditer(pattern_end, text, re.I | re.M))
    if not start_matches or not end_matches:
        return None
    candidates = []
    for start_match in start_matches:
        start_pos = start_match.start()
        for end_match in end_matches:
            end_pos = end_match.end()
            if end_pos > start_pos:
                length = len(text[start_pos:end_pos])
                if length >= minlen:
                    candidates.append((start_pos, end_pos, length))
    if not candidates:
        return None
    best = max(candidates, key=lambda x: (x[0], x[2]))
    return text[best[0] : best[1]]


def find_item(text, section, ticker):
    global ticker_pattern

    def get_patterns(section):
        if section == "Item 7":
            return {
                "strict": {
                    "start": r"^[^\S\n]*(?:(?:item)\s*7[\.:—]?\s*)?Management\W?s?\s*Discussion\s*And\s*Analysis\s*(?:of\s*(?:Financial\s*Condition\s*and\s*Results\s*of\s*Operations|Results\s*of\s*Operations?\s*and\s*Financial\s*Condition))?.?(?:\s*\(MD&A\))?[^\S\n]*$",
                    "end": r"^[^\S\n]*(?:(?:item)\s*7A[\.:—]?\s*)?(?:Quantitative\s+And\s+Qualitative|Qualitative\s+And\s+Quantitative)?\s*Disclosures?\s+About\s+(?:Market\s+)?Risk.?[^\S\n]*$",
                },
                "flexible": {
                    "start": r"(?:(?:item)\s*7[\.:—]?\s*)?Management\W?s?\s*Discussion\s*And\s*Analysis\s*(?:of\s*(?:Financial\s*Condition\s*and\s*Results\s*of\s*Operations|Results\s*of\s*Operations?\s*and\s*Financial\s*Condition))?.?(?:\s*\(MD&A\))?",
                    "end": r"(?:(?:item)\s*7A[\.:—]?\s*)?(?:Quantitative\s+And\s+Qualitative|Qualitative\s+And\s+Quantitative)?\s*Disclosures?\s+About\s+(?:Market\s+)?Risk.?",
                },
            }
        elif section == "Item 1A":
            return {
                "strict": {
                    "start": r"^[^\S\n]*(?:(?:item)\s*1A[\.:—]?\s*)?(?:Risk\s*Factors)[^\S\n]*$",
                    "end": r"^[^\S\n]*(?:(?:item)\s*1B[\.:—]?\s*)?(?:Unresolved\s*Staff\s*Comments)?[^\S\n]*$",
                },
                "flexible": {
                    "start": r"(?:(?:item)\s*1A[\.:—]?\s*)?(?:Risk\s*Factors)",
                    "end": r"(?:(?:item)\s*1B[\.:—]?\s*)?(?:Unresolved\s*Staff\s*Comments)?",
                },
            }
        else:
            raise ValueError(f"Unsupported section: {section}")

    if ticker in ticker_pattern:
        patterns = {
            "strict": {
                "start": ticker_pattern[ticker]["start"],
                "end": ticker_pattern[ticker]["end"],
            }
        }
    else:
        patterns = get_patterns(section)
    for label in ["strict", "flexible"]:
        if label not in patterns:
            continue
        start_pat = patterns[label]["start"]
        end_pat = patterns[label]["end"]
        if re.search(start_pat, text, re.I | re.M) and re.search(
            end_pat, text, re.I | re.M
        ):
            return find_mda(text, start_pat, end_pat)
    return None

In [30]:
year = 2017
filings = (
    Company("CVS")
    .get_filings()
    .filter(
        form=["10-K", "10-K/A"],
        date=f"{year}-04-01:{year+ 1}-03-31",
    )
)
filings[-1].obj()

TypeError: 'NoneType' object is not subscriptable

In [22]:
for root in roots:
    text_null = []
    text_len = []
    print(root)
    log = ""
    loop = tqdm(companies[100:])
    for ticker in loop:
        loop.set_description(f"下載 {ticker} 年報")
        searchs = [ticker] + find_cik[ticker] if ticker in find_cik else [ticker]
        for search in searchs:
            company = Company(search)
            filings = company.get_filings().filter(
                form=["10-K", "10-K/A"],
                date=f"{report_year[0]}-04-01:{report_year[1] + 1}-03-31",
            )
            if filings.empty:
                print(f"❌ {ticker} 無10-K可供下載\n")
                continue
            for filing in filings:
                try:
                    text = ""
                    report_date = datetime.fromisoformat(filing.report_date)
                    year = (
                        report_date.year - 1
                        if report_date.month <= 3
                        else report_date.year
                    )
                    loop.set_description(f"下載 {ticker} {year} 年報")
                    file_name = f"{report_folder[root]}{str(year)}/{ticker}.txt"
                    if os.path.exists(file_name) or year not in range(
                        report_year[0], report_year[1] + 1
                    ):
                        continue
                    tenk = filing.obj()
                    if not tenk:
                        print(f"❌ {ticker} {filing} 無obj\n")
                        continue
                    for x in report_item[root]:
                        item = tenk[f"Item {x}"]
                        if item:
                            item = normalize(item)
                        else:
                            item = find_item(
                                normalize(filing.text()), f"Item {x}", ticker
                            )
                            if not item:
                                continue
                        text += item + "\n"
                    if text.strip() == "":
                        text_null.append(f"{ticker}_{str(year)}")
                    else:
                        if len(text) >= minlen:
                            save_to_file(text, file_name)
                        else:
                            text_len.append(f"{ticker}_{str(year)}")
                except Exception as e:
                    print(f"❌ {file_name} 下載失敗\n{e}\n")
                    continue
    print("✅ 下載完成")
    print(log)

paper_7


下載 CVS 2016 年報:  17%|█▋        | 57/343 [02:15<13:19,  2.80s/it] 

❌ paper_7/report/2017/CVS.txt 下載失敗
'NoneType' object is not subscriptable



下載 RMD 2018 年報:  37%|███▋      | 127/343 [06:00<29:07,  8.09s/it]  

❌ paper_7/report/2019/RMD.txt 下載失敗
'NoneType' object is not subscriptable



下載 VMC 年報:  37%|███▋      | 128/343 [06:01<31:28,  8.78s/it]     

❌ paper_7/report/2018/RMD.txt 下載失敗
'NoneType' object is not subscriptable



下載 MLM 2015 年報:  39%|███▉      | 133/343 [06:20<10:00,  2.86s/it] 


KeyboardInterrupt: 

In [ ]:
# 檢查缺檔
check_files = {}
for root in roots:
    print(root)
    file_sum = 0
    check_files[root] = {}
    for ticker in companies[:companies_range]:
        for year in range(report_year[0], report_year[1] + 1):
            text = ""
            file_name = f"{report_folder[root]}{str(year)}/{ticker}.txt"
            if os.path.exists(file_name):
                with open(file_name, "r", encoding="utf-8") as f:
                    text = f.read()
                if len(text) >= minlen:
                    file_sum += 1
            if not os.path.exists(file_name) or len(text) < minlen:
                if ticker not in check_files[root]:
                    check_files[root][ticker] = []
                check_files[root][ticker].append(year)
    print(f"總計 {file_sum} 筆資料")
for root in roots:
    print(root)
    for key, value in check_files[root].items():
        print(f"{key}\t{value}")

paper_7
總計 892 筆資料
paper_17
總計 989 筆資料
paper_7
WMT	[2014, 2015, 2016]
JPM	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
XOM	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
JNJ	[2014]
CVX	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
WFC	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
AXP	[2014]
IBM	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
CAT	[2014]
VZ	[2014, 2015, 2016, 2017, 2018, 2019]
T	[2014, 2015, 2016, 2017, 2018]
RTX	[2014, 2015, 2016, 2017, 2018, 2019]
PFE	[2014, 2015, 2016, 2017, 2018, 2019]
PGR	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
DE	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
SO	[2014, 2015, 2016, 2017]
SHW	[2014, 2015, 2016, 2017, 2018]
paper_17
JNJ	[2014]
WFC	[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [24]:
# CIK補10K
for root in roots:
    print(root)
    log = ""
    keys = list(check_files[root].keys())
    loop = tqdm(keys)
    for ticker in loop:
        try:
            loop.set_description(
                f"補齊 {ticker} {', '.join(str(y) for y in years)}年 檔案"
            )
            years = check_files[root][ticker].copy()
            searchs = [ticker] + find_cik.get(ticker, [])
            for search in searchs:
                loop.set_postfix_str(
                    f"以 {search:<11} 搜尋 {ticker} {year}年 檔案中..."
                )
                filings = Company(search).get_filings(
                    form=["10-K", "10-K/A"],
                    date=f"{report_year[0]}-01-01:{report_year[1] + 1}-12-31",
                )
                for filing in filings:
                    text = ""
                    report_date = datetime.fromisoformat(filing.report_date)
                    year = (
                        report_date.year - 1
                        if report_date.month <= 3
                        else report_date.year
                    )
                    file_name = f"{report_folder[root]}{str(year)}/{ticker}.txt"
                    if os.path.exists(file_name) or year not in years:
                        continue
                    loop.set_postfix_str(f"下載 {ticker} {year}年 檔案中...")
                    for x in report_item[root]:
                        tenk = filing.obj()
                        item = tenk[f"Item {x}"]
                        if not item:
                            item = find_item(filing.text(), f"Item {x}", ticker)
                            if not item:
                                continue
                        text += item + "\n"
                    if text.strip() == "":
                        log += f"⚠️ {file_name} 無文本。\n"
                    else:
                        text = normalize(text)
                        if len(text) >= minlen:
                            save_to_file(text, file_name)
                            log += f"✅ {file_name} 下載完成\n"
                        else:
                            log += f"⚠️ {file_name} 不足規定字數。\n"
                    years.remove(year)
        except Exception as e:
            log += f"❌ {file_name} 下載失敗\n{e}\n"
    print(log)

paper_7


補齊 JPM 年 檔案:   6%|▌         | 1/17 [00:25<06:45, 25.34s/it, 下載 JPM 2017年 檔案中...]               


KeyboardInterrupt: 

In [32]:
Company("WM")

╭────────────────────────────────────── 🏢 WASTE MANAGEMENT INC [823768] WM ──────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ ╭───────────────────────────────────────────────── 📋 Entity ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │   CIK   823768   Type   Operating   ○                                                                       │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │   Category                  Industry               Fiscal Year End    

In [ ]:
# 查單檔
ticker = "WM"
years = [2018]
company = Company("WM")  # Microsoft

# 4. Get company filings
filings = (
    company.get_filings()
    .filter()
    .filter(
        form=["10-K", "10-K/A"],
        date=f"{report_year[0]}-04-01:{report_year[1] + 1}-03-31",
    )
)  # Insider transactions

# 6. Get the latest filing
insider_filing = filings[0]

# 7. Convert to a data object
ownership = insider_filing.obj()
ownership

╭─────────────────────────────────────────── WASTE MANAGEMENT INC 10-K ───────────────────────────────────────────╮
│ Period ending December 31, 2023 filed on February 13, 2024                                                      │
│                                                                                                                 │
│                                                                                                                 │
│ 📄                                                                                                              │
│ ├── PART I                                                                                                      │
│ │   ├── Item 1  Business                                                                                        │
│ │   ├── Item 1A Risk Factors                                                                                    │
│ │   ├── Item 1B Unresolved Staff Comments                              

In [ ]:
# 排序 companies
sp500_tickers = list(df_companies.Symbol.values)
with open("companies.json", "r") as f:
    data = json.load(f)
find_df = pd.DataFrame(data["find_cik"])
find_df["sp500_rank"] = find_df["ticker"].apply(
    lambda x: sp500_tickers.index(x) if x in sp500_tickers else float("inf")
)
find_df = find_df.sort_values("sp500_rank").drop(columns="sp500_rank")
sorted_find_companies = {"find_cik": find_df.to_dict(orient="records")}
missing_df = pd.DataFrame(data["missing_companies"])
missing_df["sp500_rank"] = missing_df["ticker"].apply(
    lambda x: sp500_tickers.index(x) if x in sp500_tickers else float("inf")
)
missing_df = missing_df.sort_values("sp500_rank").drop(columns="sp500_rank")
sorted_companies = {
    "find_cik": find_df.to_dict(orient="records"),
    "missing_companies": missing_df.to_dict(orient="records"),
}
output_path = "companies.json"
with open(output_path, "w") as f:
    json.dump(sorted_companies, f, indent=2, ensure_ascii=False)

In [292]:
# 排序 ticker_pattern
sp500_tickers = list(df_companies.Symbol.values)
pattern_list = [ticker for ticker in sp500_tickers if ticker in ticker_pattern.keys()]
pattern_list

['GOOGL', 'AVGO', 'DIS', 'BLK', 'CI', 'TPL', 'FOXA', 'STE', 'BG', 'WBA', 'APA']

In [317]:
# 排序 find_cik
sp500_tickers = list(df_companies.Symbol.values)
find_list = [ticker for ticker in sp500_tickers if ticker in find_cik.keys()]
find_list

['GOOGL', 'AVGO', 'DIS', 'BLK', 'CI', 'TPL', 'FOXA', 'STE', 'BG', 'WBA', 'APA']

In [ ]:
# 找出每家公司第一次出現空值的年份與第一次有檔案的年份
def find_first_empty_and_first_nonempty(row):
    years = df.columns[1:]  # 第一欄應該是 ticker
    first_empty_year = None
    first_nonempty_year = None
    for year in years:
        value = row[year].strip()
        if value == "" or value.lower() == "nan":
            if first_empty_year is None:
                first_empty_year = int(year)
        else:
            if first_nonempty_year is None:
                first_nonempty_year = int(year)
    return pd.Series([first_empty_year, first_nonempty_year])


# 套用到所有公司
df[["first_empty", "first_nonempty"]] = df.apply(
    find_first_empty_and_first_nonempty, axis=1
)
# 篩選：只有當 first_empty > first_nonempty 才表示是中間缺資料（有缺但不是因為上市年份）
missing_companies = df[df["first_empty"] > df["first_nonempty"]]
missing_companies

In [281]:
Company("FCX").get_filings(form=["10-K", "10-K/A"]).filter(date="2014-01-01:2024-12-31")

╭─────────────────────────────────── Filings for FREEPORT-MCMORAN INC [831259] ───────────────────────────────────╮
│                                                                                                                 │
│        form     filed        accession_number       xbrl                                                        │
│  ────────────────────────────────────────────────────────                                                       │
│   0    10-K     2024-02-16   0000831259-24-000011   ✓                                                           │
│   1    10-K     2023-02-15   0000831259-23-000013   ✓                                                           │
│   2    10-K/A   2022-02-18   0000831259-22-000012   ✓                                                           │
│   3    10-K     2022-02-15   0000831259-22-000009   ✓                                                           │
│   4    10-K     2021-02-16   0000831259-21-000009   ✓                 

In [ ]:
year = 2017
filings = Company(ticker).get_filings(
    form="10-K", date=f"{str(year)}-01-01:{str(year + 1)}-12-31"
)
print(filings[0].text())